In [324]:
import numpy as np 
import pandas as pd
import csv 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.tree import DecisionTreeClassifier
import os 

os.chdir("/Users/susanchen/Documents/GitHub/skincare_classification/Plots")
from collections import Counter 

## Building Multi-label Classifiers 
### Base Classifiers to test: Binary Relevance (no correlation), Binary Relevance (with correlation), Multi-label Random Forest, Multi-label Decision Tree, Classifier Chain, Binary Releveance with Stacking Aggregation, Cross-Coupling Aggregation (to combat class-imbalances)

In [670]:
data = pd.read_csv('/Users/susanchen/Documents/GitHub/skincare_classification/Data_sets/Master_cleaned.csv')

In [834]:
smallClasses = ["['Oily', 'Sensitive']", "['Dry', 'Normal', 'Oily']", "['Dry', 'Oily']", "['Dry', 'Normal', 'Oily', 'Sensitive']"]
data = data[data.Skin_Type != "['Oily', 'Sensitive']"]
data = data[data.Skin_Type != "['Dry', 'Normal', 'Oily']"]
data = data[data.Skin_Type != "['Dry', 'Oily']"]
data = data[data.Skin_Type != "['Dry', 'Normal', 'Oily', 'Sensitive']"]
data = data[data.Skin_Type != "['Combination', 'Dry', 'Oily', 'Sensitive']"]
data = data[data.Skin_Type != "['Normal', 'Oily']"]

X = data[data.columns[12:23]].values
y = data.Skin_Type

In [835]:
X = data[data.columns[12:23]].values
y = data.Skin_Type
#['Oily', 'Sensitive']": 2, "['Dry', 'Normal', 'Oily']": 2, "['Dry', 'Oily']": 1, "['Dry', 'Normal', 'Oily', 'Sensitive']": 1
#"['Combination', 'Dry', 'Oily', 'Sensitive']": 4, "['Normal', 'Oily']": 4
#summarize class distribution 
counter = Counter(y)
print(counter)



Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 918, "['Combination', 'Dry', 'Normal', 'Oily']": 239, "['Combination', 'Normal', 'Oily']": 113, "['Combination', 'Oily']": 91, "['Normal']": 83, "['Dry', 'Normal']": 72, "['Dry']": 71, "['Combination', 'Dry', 'Normal']": 66, "['Combination', 'Dry', 'Normal', 'Sensitive']": 61, "['Combination']": 59, "['Sensitive']": 46, "['Combination', 'Normal']": 30, "['Dry', 'Normal', 'Sensitive']": 29, "['Oily']": 26, "['Combination', 'Normal', 'Oily', 'Sensitive']": 23, "['Combination', 'Dry']": 22, "['Dry', 'Sensitive']": 19, "['Combination', 'Dry', 'Oily']": 13, "['Normal', 'Sensitive']": 12, "['Combination', 'Oily', 'Sensitive']": 10, "['Combination', 'Dry', 'Sensitive']": 7})


In [836]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer 

X = data[data.columns[12:23]].values
y = data["Skin_Type"].to_numpy()


X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, shuffle=True, test_size=0.3, random_state = 1024)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, shuffle=False, test_size=0.5, random_state = 1024)

print("Dataset sizes:\n\tTrain %s\n\tVal %s\n\tTest %s" % (X_train.shape, X_val.shape,X_test.shape))

Dataset sizes:
	Train (1407, 11)
	Val (301, 11)
	Test (302, 11)


In [837]:
#SMOTE
# transform the dataset
#['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 652, "['Combination', 'Dry', 'Normal', 'Oily']": 166
num = 100
num2 = 50
# summarize the class distribution
counter = Counter(y_train)
print("Before resampling:") 
print (counter)
oversample = SMOTE(k_neighbors = 3, sampling_strategy = {"['Combination', 'Normal', 'Oily']": num, "['Combination', 'Oily']": num, "['Dry']": num, "['Normal']": num, "['Combination', 'Dry', 'Normal', 'Sensitive']": num, "['Dry', 'Normal']": num, "['Combination', 'Dry', 'Normal']": num, "['Combination']": num, "['Sensitive']": num2, "['Combination', 'Normal']": num2, "['Dry', 'Normal', 'Sensitive']": num2, "['Dry', 'Sensitive']": num2, "['Combination', 'Dry']": num2, "['Oily']": num2, "['Combination', 'Normal', 'Oily', 'Sensitive']": num2, "['Combination', 'Dry', 'Oily']": num2, "['Normal', 'Sensitive']": num2, "['Combination', 'Oily', 'Sensitive']": num2, "['Combination', 'Dry', 'Sensitive']": num2})
X_train, y_train = oversample.fit_resample(X_train, y_train)

# summarize the new class distribution
#under = RandomUnderSampler(sampling_strategy= {"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 600})
#X_train, y_train = under.fit_resample(X_train, y_train)

counter = Counter(y_train)
print()
print("After resampling:") 
print(counter)

Before resampling:
Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 652, "['Combination', 'Dry', 'Normal', 'Oily']": 166, "['Combination', 'Normal', 'Oily']": 81, "['Combination', 'Oily']": 71, "['Dry']": 53, "['Normal']": 53, "['Combination', 'Dry', 'Normal', 'Sensitive']": 45, "['Dry', 'Normal']": 43, "['Combination', 'Dry', 'Normal']": 41, "['Combination']": 40, "['Sensitive']": 29, "['Combination', 'Normal']": 22, "['Dry', 'Normal', 'Sensitive']": 17, "['Dry', 'Sensitive']": 17, "['Combination', 'Dry']": 16, "['Oily']": 16, "['Combination', 'Normal', 'Oily', 'Sensitive']": 14, "['Combination', 'Dry', 'Oily']": 9, "['Normal', 'Sensitive']": 9, "['Combination', 'Oily', 'Sensitive']": 7, "['Combination', 'Dry', 'Sensitive']": 6})

After resampling:
Counter({"['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']": 652, "['Combination', 'Dry', 'Normal', 'Oily']": 166, "['Combination', 'Dry', 'Normal', 'Sensitive']": 100, "['Combination', 'Dry', 'Normal']": 100, "['Combi

In [838]:
print("Dataset sizes:\n\tTrain %s\n\tVal %s\n\tTest %s" % (X_train.shape, X_val.shape,X_test.shape))

Dataset sizes:
	Train (2168, 11)
	Val (301, 11)
	Test (302, 11)


In [839]:
y_train

array(["['Combination', 'Dry', 'Normal', 'Oily']",
       "['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']",
       "['Combination', 'Dry', 'Normal', 'Sensitive']", ...,
       "['Sensitive']", "['Sensitive']", "['Sensitive']"], dtype=object)

In [840]:
for i in range(len(y_train)):
    y_train[i] = [y_train[i]]

In [841]:
mlb = MultiLabelBinarizer(classes = ('Combination', 'Dry', 'Normal', 'Oily', 'Sensitive'))
y_train = mlb.fit_transform(y_train)

#a = mlb.fit_transform(set(data['Skin_Type'].str.split(', '))) 
#y_val = mlb.fit_transform(y_val)
#y_test = mlb.fit_transform(y_test)

In [842]:
mlb.classes_

array(['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive'], dtype=object)

In [845]:
y_train[:10]

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

## Binary Relevance with Gaussian 

In [120]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
# initialize binary relevance multi-label classifier with a gaussian naive bayes base classifier
classifier = BinaryRelevance(
    classifier = GaussianNB(),
    require_dense=[True, True] 
)
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_val)
# accuracy
print("Accuracy = ", metrics.accuracy_score(y_val,predictions))
print("F1 Score = ", metrics.f1_score(y_val,predictions, average = "micro"))
print("Hamming Loss", metrics.hamming_loss(y_val, predictions))


Accuracy =  0.14950166112956811
F1 Score =  0.7266277128547579
Hamming Loss 0.3784486494294381


## One vs the Rest Classifier 

In [121]:
from sklearn.multiclass import OneVsRestClassifier
classif = OneVsRestClassifier(SVC(kernel='linear', random_state= 1024))
classif.fit(X_train, y_train)
val_pred = classif.predict(X_val)
# accuracy
print("Accuracy = ", metrics.accuracy_score(y_val, val_pred))
print("F1 Score = ", metrics.f1_score(y_val,val_pred, average = "micro"))
print("Hamming Loss", metrics.hamming_loss(y_val, val_pred))

Accuracy =  0.2956810631229236
F1 Score =  0.8926022336048367
Hamming Loss 0.18474649718330205


## Decision Tree Classifier

In [122]:
from sklearn.model_selection import GridSearchCV
dt =  DecisionTreeClassifier()
param_grid = {
    'max_depth' :[5,6,7,8,9],
    'max_leaf_nodes': [5,10,15]
}
gs = GridSearchCV(dt, param_grid, scoring = "accuracy", cv= 5)

gs.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'max_leaf_nodes': [5, 10, 15]},
             scoring='accuracy')

In [123]:
tree = dt.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(metrics.f1_score(y_test, y_pred, average="micro"))

0.8405563032750112


  ## Binary Relevance with Random Forest Classifier 

In [124]:
from sklearn.ensemble import RandomForestClassifier
import time 
start=time.time()
classifier = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')


training time taken:  14.0 seconds


In [125]:
y_hat=classifier.predict(X_test)
a=metrics.accuracy_score(y_test, y_hat)
br_f1=metrics.f1_score(y_test, y_hat, average='micro')
br_hamm=metrics.hamming_loss(y_test,y_hat)

print('Binary Relevance accuracy:',round(a,3))
print('Binary Relevance F1-score:',round(br_f1,3))
print('Binary Relevance Hamming Loss:',round(br_hamm,3))

Binary Relevance accuracy: 0.252
Binary Relevance F1-score: 0.881
Binary Relevance Hamming Loss: 0.201


## Label Powerset with Random Forest Classifier

In [126]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

start=time.time()
classifier.fit(X_train, y_train)
print('training time taken: ',round(time.time()-start,0),'seconds')

training time taken:  1.0 seconds


In [127]:
y_hat=classifier.predict(X_test)
lp_a=metrics.accuracy_score(y_test, y_hat)
lp_f1=metrics.f1_score(y_test, y_hat, average='micro')
lp_hamm=metrics.hamming_loss(y_test,y_hat)
print('Binary Relevance accuracy:',round(lp_a,3))
print('Label Powerset F1-score:',round(lp_f1,3))
print('Label Powerset Hamming Loss:',round(lp_hamm,3))

Binary Relevance accuracy: 0.334
Label Powerset F1-score: 0.87
Label Powerset Hamming Loss: 0.219


## Multi-label K Nearest Neighbors

In [78]:
from skmultilearn.adapt import MLkNN
import time

parameters = {'k': range(1,3), 
              's': [0.5, 0.7, 1.0]}

score = 'f1_micro'

start=time.time()

classifier = GridSearchCV(MLkNN(), parameters, scoring=score)
classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')
print('best parameters :', classifier.best_params_, 'best score: ',
      classifier.best_score_)

training time taken:  86.0 seconds
best parameters : {'k': 2, 's': 0.5} best score:  0.8640679016137314


## 

## Binary Relevance K-Nearest Neighbors

In [128]:
from skmultilearn.adapt import BRkNNaClassifier

parameters = {'k': range(3,5)}
score = 'f1_samples'

start=time.time()

classifier = GridSearchCV(BRkNNaClassifier(), parameters, scoring=score)
classifier.fit(X_train, y_train)

print('training time taken: ',round(time.time()-start,0),'seconds')
print('best parameters :', classifier.best_params_,
      'best score: ',classifier.best_score_)


training time taken:  2.0 seconds
best parameters : {'k': 3} best score:  0.8347857185580414


In [135]:
BrKnn = BRkNNaClassifier(k= 3)
BrKnn.fit(X_train, y_train)
pred = BrKnn.predict(X_test)
print(metrics.accuracy_score(y_test, pred))
print(metrics.f1_score(y_test, pred, average='micro'))

0.24834437086092714
0.8625294579732913


In [81]:
import math

import pandas as pd
import numpy as np
import scipy.io as sio

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.cluster import KMeans
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

In [206]:
ratio= 0.1

def mLIFT(train_data,train_target,test_data,test_target,ratio):
    num_train, dim= train_data.shape
    num_test, num_class= test_target.shape
    
    P_Centers= []
    N_Centers= []
    
    ##### KMeans, and save the centers
    for i in range(num_class):
        print ("Performing clustering:%d/%d" %(i+ 1, num_class))
    
        p_data= train_data[train_target[:,i]== 1]
        n_data= train_data[train_target[:,i]== -1]
    
        k1= int(min( math.ceil(p_data.shape[0]* ratio), math.ceil(n_data.shape[0]* ratio) ))
        #print("k1= k2= %d" %k1)
        k2= k1;
    
        if(k1== 0):
            POS_C=[]
            zero_kmeans= KMeans(n_clusters= min(50, num_train)).fit(train_data)
            NEG_C= zero_kmeans.cluster_centers_
        else:
            # Positive
            if(p_data.shape[0]== 1):
                POS_C= p_data
            else:
                p_kmeans= KMeans(n_clusters= k1).fit(p_data)
                POS_C= p_kmeans.cluster_centers_
            # Negative
            if(n_data.shape[0]== 1):
                NEG_C= n_data
            else:
                n_kmeans= KMeans(n_clusters= k2).fit(n_data)
                NEG_C= n_kmeans.cluster_centers_
                
        # Save the cluster centers
        P_Centers.append(POS_C)
        N_Centers.append(NEG_C)
            
    #print("The size of P_Canters is %d\n" %len(P_Centers))
    
    ##### Do the map and save the models
    Models= []
    for i in range(num_class):
        print ("Building classifiers: :%d/%d" %(i+ 1, num_class))
        centers= np.vstack((P_Centers[i+1], N_Centers[i+1]))
        num_center= centers.shape[0]
        # print(num_center)
        data= []
    
        if(num_center>= 5000):
            print("Too many cluster center!")
            break
        else:
            blocksize= 5000- num_center
            num_block= int(math.ceil(num_train/ blocksize))
            # print(num_block)
            
            mFirst= True
            for j in range(num_block- 1):
                print(j)
                low= j* blocksize
                high= (j+ 1)* blocksize
                # Calculate the distance
                for k in range(num_center):
                    diff= train_data[low:high, :]- centers[k]
                    Eu_diff= np.linalg.norm(diff, axis=1)
                    if(mFirst== True):
                        mFirst= False
                        data_temp= Eu_diff
                    else:
                        data_temp= np.vstack((data_temp, Eu_diff))
                    
            
            low= (num_block- 1)* blocksize
            high= num_train
            
            # Calculate the distance
            for j in range(num_center):
                diff= train_data[low:high,:]- centers[j]
                Eu_diff= np.linalg.norm(diff, axis=1)
                if(mFirst== True):
                    mFirst= False
                    data_temp= Eu_diff
                else:
                    data_temp= np.vstack((data_temp, Eu_diff))
            
            data= data_temp.T
        
        training_instance_matrix= data
        training_label_vector= train_target[:,i]
    
        model_this= SVC(C= 10, probability=True).fit(training_instance_matrix, training_label_vector)
        #model_this= LogisticRegression(C= 0.03).fit(training_instance_matrix, training_label_vector)
        #model_this= DecisionTreeClassifier().fit(training_instance_matrix, training_label_vector)
        #model_this = AdaBoostClassifier(DecisionTreeClassifier(),
                         #algorithm="SAMME",
                         #n_estimators=50, learning_rate=0.8).fit(training_instance_matrix, training_label_vector)
        Models.append(model_this)
    
    ##### Predict
    for i in range(num_class):
        print ("Predicting: :%d/%d" %(i+ 1, num_class))
        centers= np.vstack((P_Centers[i], N_Centers[i]))
        num_center= centers.shape[0]
        # print(num_center)
        data= []
    
        if(num_center>= 5000):
            print("Too many cluster center!")
            break
        else:
            blocksize= 5000- num_center
            num_block= int(math.ceil(num_test/ blocksize))
            # print(num_block)
            
            mFirst= True
            for j in range(num_block- 1):
                print(j)
                low= j* blocksize
                high= (j+ 1)* blocksize
                # Calculate the distance
                for k in range(num_center):
                    diff= test_data[low:high, :]- centers[k]
                    Eu_diff= np.linalg.norm(diff, axis=1)
                    if(mFirst== True):
                        mFirst= False
                        data_temp= Eu_diff
                    else:
                        data_temp= np.vstack((data_temp, Eu_diff))
                    
            
            low= (num_block- 1)* blocksize
            high= num_train
            
            # Calculate the distance
            for j in range(num_center):
                diff= test_data[low:high,:]- centers[j]
                Eu_diff= np.linalg.norm(diff, axis=1)
                if(mFirst== True):
                    mFirst= False
                    data_temp= Eu_diff
                else:
                    data_temp= np.vstack((data_temp, Eu_diff))
            
            data= data_temp.T
            # print(data.shape)
    
        testing_instance_matrix= data;
        testing_label_vector= test_target[:, i]
    
        predicted_label= Models[i].predict(testing_instance_matrix)
        
        #print (predicted_label)

        print("The accuracy is: %f" %accuracy_score(testing_label_vector, predicted_label))
        #print(roc_auc_score(testing_label_vector, predicted_label))
    
    return 1


In [208]:
mLIFT(X_train,y_train, X_test,y_test,ratio);

ValueError: not enough values to unpack (expected 2, got 1)

In [162]:
print(P_Centers)

NameError: name 'P_Centers' is not defined

In [173]:
p_data= X_train[y_train== 1]

IndexError: boolean index did not match indexed array along dimension 1; dimension is 11 but corresponding boolean dimension is 23

In [170]:
X_train.shape

(2168, 11)

In [169]:
y_train.shape

(2168, 23)